# Convolutional Neural Network
using a pretrained model

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models


from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

import time

### Defining a model

In [5]:
import torch
import torch.nn as nn
from torchvision import models

class MobileNetV2MNIST(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()

        self.model = models.mobilenet_v2(pretrained=pretrained)

        self.model.features[0][0] = nn.Conv2d(
            1, 32, kernel_size=3, stride=2, padding=1, bias=False
        )

        self.model.classifier[1] = nn.Linear(
            self.model.classifier[1].in_features, 10
        )

    def forward(self, x):
        return self.model(x)


### Transformation and train-test split

In [7]:
transform = transforms.Compose([
    transforms.Resize(96),   # 96x96 wystarczy
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

### Training

In [8]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
model = MobileNetV2MNIST(pretrained=True)

for p in model.model.features.parameters():
    p.requires_grad = False

In [10]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3
)

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

model = MobileNetV2MNIST(pretrained=True).to(device)


In [13]:
def train(model, train_loader, test_loader, epochs):
    for epoch in range(epochs):
        # ===== TRAIN =====
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # ===== EVAL =====
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                preds = outputs.argmax(dim=1)

                correct += (preds == labels).sum().item()
                total += labels.size(0)

        acc = 100 * correct / total

        print(
            f"Epoch [{epoch+1}/{epochs}] "
            f"Loss: {running_loss/len(train_loader):.4f} "
            f"Test Acc: {acc:.2f}%"
        )


In [ ]:
train(model, train_loader, test_loader, epochs=5)

Epoch [1/5] Loss: 0.3995 Test Acc: 92.80%
